# Tensorflow

Learning clustering functions, half following FCC tutorial/tensorflow docs

In [3]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf
import tensorflow_datasets as tfds

cur_dir = os.getcwd()
tf.__version__

'2.7.0'

In [48]:
train_ds, test_ds = tfds.load('Iris', split=['train[:80%]', 'train[80%:]'], data_dir=cur_dir)

print('Size of training dataset: ', len(train_ds), '\n', 'Size of validation set: ', len(test_ds))

Size of training dataset:  120 
 Size of validation set:  30


In [49]:
# Convert them to dataframes 
train_ds = tfds.as_dataframe(train_ds)
test_ds = tfds.as_dataframe(test_ds)


In [50]:
# Show the format of the dataset
# For some reason it just gives us a list in features, with no names to said features
train_ds.head()

,features,label
0,"[5.1, 3.4, 1.5, 0.2]",0
1,"[7.7, 3.0, 6.1, 2.3]",2
2,"[5.7, 2.8, 4.5, 1.3]",1
3,"[6.8, 3.2, 5.9, 2.3]",2
4,"[5.2, 3.4, 1.4, 0.2]",0


In [51]:
# I'm just assuming these are the feature names as commonly returned online
# TFDS documentation doesn't actually state what they are (how annoying)
feature_names = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"]

train_labels = train_ds.pop('label')
train_ds = pd.DataFrame(train_ds['features'].to_list(), columns=feature_names) # Convert column of lists to actual columns

train_ds.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.1,3.4,1.5,0.2
1,7.7,3.0,6.1,2.3
2,5.7,2.8,4.5,1.3
3,6.8,3.2,5.9,2.3
4,5.2,3.4,1.4,0.2


In [52]:
# Repeat for test dataset
test_labels = test_ds.pop('label')
test_ds = pd.DataFrame(test_ds['features'].to_list(), columns=feature_names) # Convert column of lists to actual columns

In [53]:
# https://www.tensorflow.org/tutorials/estimator/premade#create_input_functions
# https://www.tensorflow.org/guide/data
def input_function(features, labels, training=True, batch_size=32):
    # Convert to a dataset for Tensorflow to accept
    # https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices for future reference
    # Format of those going into the tensors - tuple ({'SepalLength': [...(array containing Sepal Length)], 'SepalWidth': [...], ...}, label)
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [54]:
# Define feature columns
feature_columns = []
for key in train_ds.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key)) # All our features are numeric this time

In [55]:
feature_columns # See what's stored in our features

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [56]:
# Build a deep neural network model on top of Tensorflow
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # Create two hidden layers in our NN with 30 and 10 nodes respectively
    hidden_units=[30,10],
    # We had 3 species to classify
    n_classes=3
)

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpesq7tkyc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpesq7tkyc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [57]:
# Train said model
classifier.train(
    input_fn=lambda: input_function(train_ds, train_labels, training=True), # use our input_function defined above
    steps=5000
)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Create CheckpointSaverHook.


...

INFO:tensorflow:Saving checkpoints for 5000 into C:\Users\User\AppData\Local\Temp\tmpesq7tkyc\model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...


INFO:tensorflow:Loss for final step: 0.47725087.


INFO:tensorflow:Loss for final step: 0.47725087.


In [58]:
# Not sure why my loss is so high...
# Let's check the evaluation
# Code taken from tensorflow docs
eval_result = classifier.evaluate(
    input_fn=lambda: input_function(test_ds, test_labels, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Starting evaluation at 2021-11-25T20:25:05


...

INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8666667, average_loss = 0.55771095, global_step = 5000, loss = 0.55771095


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8666667, average_loss = 0.55771095, global_step = 5000, loss = 0.55771095


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\User\AppData\Local\Temp\tmpesq7tkyc\model.ckpt-5000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\User\AppData\Local\Temp\tmpesq7tkyc\model.ckpt-5000



Test set accuracy: 0.867



In [102]:
# Yeah the accuracy is pretty bad, but oh well.
# Let's try predicting with a bunch of random values 
# I just yoinked these numbers off the docs
species = ['Setosa', 'Versicolor', 'Viriginica'] # Actual labels
to_predict = {    
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
    }


def pred_input(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

# Instantiate classifier for predictions
# The predict method returns a Python iterable, yielding a dictionary of prediction results for each example.
predictions = classifier.predict(
    input_fn=lambda: pred_input(to_predict)
)

In [81]:
# peekin at what's in the predictions variable..
predictions

<generator object Estimator.predict at 0x000001485B356D60>

In [103]:
for pred, expect in zip(predictions, species):
    print(pred)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


{'logits': array([-0.02572348, -1.4914739 , -2.3360164 ], dtype=float32), 'probabilities': array([0.7518024 , 0.17359465, 0.07460298], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
{'logits': array([-3.5112658, -2.7144306, -2.887805 ], dtype=float32), 'probabilities': array([0.19670013, 0.43638092, 0.366919  ], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
{'logits': array([-4.9693136, -3.8094409, -3.2305226], dtype=float32), 'probabilities': array([0.10121471, 0.3228271 , 0.5759582 ], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}


In [101]:
for pred, expect in zip(predictions, species):
    class_id = pred['class_ids'][0] # Gets the ID of the class it's predicting to be
    probability = pred['probabilities'][class_id] # Gets the probability that the estimator would have guessed this class

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        species[class_id], 100 * probability, expect))


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Graph was finalized.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Prediction is "Setosa" (75.2%), expected "Setosa"
Prediction is "Versicolor" (43.6%), expected "Versicolor"
Prediction is "Viriginica" (57.6%), expected "Viriginica"
